In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
import os
os.makedirs("./model_parameters", exist_ok=True)

In [3]:
device='cuda'if torch.cuda.is_available()else'cpu'

In [4]:
torch.manual_seed(1)
if device=='cuda':
    torch.cuda.manual_seed_all(1)

In [5]:
trans=transforms.Compose([
    transforms.ToTensor()
])

In [6]:
train_data=dsets.ImageFolder(root='./custom_data/train_data',transform=trans)

In [7]:
data_loader=torch.utils.data.DataLoader(dataset=train_data,
                                        shuffle=True,
                                        drop_last=True,
                                        batch_size=8,
                                        num_workers=2)


In [8]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #(8,3,64,128)
        self.layer1=torch.nn.Sequential(
        torch.nn.Conv2d(3,6,5),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2))
        #(8,6,30,62)
        self.layer2=torch.nn.Sequential(
        torch.nn.Conv2d(6,16,5),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2))
        #(8,16,13,29)
        self.fc1=torch.nn.Linear(16*13*29,120)
        self.fc2=torch.nn.Linear(120,2)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        self.relu=torch.nn.ReLU()
        
    def forward(self,x):
        out1=self.layer1(x)
        out2=self.layer2(out1)
        out3=self.fc1(out2.view(-1,16*13*29))
        out4=self.relu(out3)
        out5=self.fc2(out4)
        return out5

In [9]:
model=CNN().to(device)

test_sample=torch.Tensor(3,3,64,128).to(device)
test_out=model(test_sample)
print(test_out.shape)

optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)
criterion=torch.nn.CrossEntropyLoss().to(device)
total_batch=len(data_loader)

torch.Size([3, 2])


In [10]:
for epoch in range(10):
    avg_cost=0
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        hypothesis=model(X)
        loss=criterion(hypothesis,Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_cost+=loss/total_batch
    print("[Epoch:{:2d}],[Cost:{:.9f}]".format((epoch+1),avg_cost))
print("Finished!")

[Epoch: 1],[Cost:0.322520494]
[Epoch: 2],[Cost:0.030967999]
[Epoch: 3],[Cost:0.008000401]
[Epoch: 4],[Cost:0.003821855]
[Epoch: 5],[Cost:0.002309267]
[Epoch: 6],[Cost:0.001165453]
[Epoch: 7],[Cost:0.000847009]
[Epoch: 8],[Cost:0.000511847]
[Epoch: 9],[Cost:0.000360981]
[Epoch:10],[Cost:0.000284599]
Finished!


In [11]:
trans = transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = dsets.ImageFolder(root='./custom_data/test_data', transform=trans)
test_set = torch.utils.data.DataLoader(dataset=test_data, batch_size=len(test_data))

with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = model(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())


Accuracy: 1.0
